# 1. Загрузка набора данных

In [17]:
import sys

print(sys.version)

import pkg_resources

def get_installed_packages():
    installed_packages = pkg_resources.working_set
    packages = sorted(["%s==%s" % (i.key, i.version)
                       for i in installed_packages])
    return packages

if __name__ == "__main__":
    packages = get_installed_packages()
    if packages:
        for package in packages:
            print(package)

3.10.12 (main, Feb  4 2025, 14:57:36) [GCC 11.4.0]
anyio==4.8.0
argon2-cffi-bindings==21.2.0
argon2-cffi==23.1.0
arrow==1.3.0
asttokens==3.0.0
async-lru==2.0.4
attrs==25.1.0
babel==2.17.0
beautifulsoup4==4.13.3
bleach==6.2.0
certifi==2025.1.31
cffi==1.17.1
charset-normalizer==3.4.1
comm==0.2.2
debugpy==1.8.13
decorator==5.2.1
defusedxml==0.7.1
exceptiongroup==1.2.2
executing==2.2.0
fastjsonschema==2.21.1
fqdn==1.5.1
h11==0.14.0
httpcore==1.0.7
httpx==0.28.1
idna==3.10
ipykernel==6.29.5
ipython==8.33.0
ipywidgets==8.1.5
isoduration==20.11.0
jedi==0.19.2
jinja2==3.1.5
json5==0.10.0
jsonpointer==3.0.0
jsonschema-specifications==2024.10.1
jsonschema==4.23.0
jupyter-client==8.6.3
jupyter-console==6.6.3
jupyter-core==5.7.2
jupyter-events==0.12.0
jupyter-lsp==2.2.5
jupyter-server-terminals==0.5.3
jupyter-server==2.15.0
jupyter==1.1.1
jupyterlab-pygments==0.3.0
jupyterlab-server==2.27.3
jupyterlab-widgets==3.0.13
jupyterlab==4.3.5
markupsafe==3.0.2
matplotlib-inline==0.1.7
mistune==3.1.2
nbcli

In [11]:
%%capture
%wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [2]:
from corus import load_lenta
import pandas as pd

path = 'lenta-ru-news.csv.gz'
records = load_lenta(
    path)

data = []
for record in records:
    if record:  # Убедитесь, что запись не None
        data.append({
            'title': record.title,
            'text': record.text,
            'topic': record.topic
        })
df = pd.DataFrame(data)

## 2. Подготовка данных

In [3]:
df['topic'].value_counts()

topic
Россия               160519
Мир                  136680
Экономика             79538
Спорт                 64421
Культура              53803
Бывший СССР           53402
Наука и техника       53136
Интернет и СМИ        44675
Из жизни              27611
Дом                   21734
Силовые структуры     19596
Ценности               7766
Бизнес                 7399
Путешествия            6408
69-я параллель         1268
Крым                    666
Культпросвет            340
                        203
Легпром                 114
Библиотека               65
Оружие                    3
ЧМ-2014                   2
МедНовости                1
Сочи                      1
Name: count, dtype: int64

In [4]:
df['topic'] = df['topic'].replace('', 'EMPTY')

Убираю низкочисленные классификаторы, потому что их влияние будет ровно 0 и количество строк у классификатора, должно быть больше 1.

In [5]:
topic_counts = df['topic'].value_counts()
min_samples = 5
class_df = df[df['topic'].isin(topic_counts[topic_counts >= min_samples].index)]

class_df['topic'].value_counts()

topic
Россия               160519
Мир                  136680
Экономика             79538
Спорт                 64421
Культура              53803
Бывший СССР           53402
Наука и техника       53136
Интернет и СМИ        44675
Из жизни              27611
Дом                   21734
Силовые структуры     19596
Ценности               7766
Бизнес                 7399
Путешествия            6408
69-я параллель         1268
Крым                    666
Культпросвет            340
EMPTY                   203
Легпром                 114
Библиотека               65
Name: count, dtype: int64

На данном этапе, после получение accuracy_score 0,82 на тестовой выборке, я захотел проэксперементировать и постараться распределить весы путем попытки равномерного распределения классификаторов и посмотреть как это может улучшить результат после обучения модели.

In [5]:
import pandas as pd

n_classes = class_df['topic'].nunique()
target_per_class = 100_000 // n_classes  # Базовый таргет на класс

# Собираем данные поровну, не превышая исходное количество
balanced_dfs = []
for topic in class_df['topic'].unique():
    subset = class_df[class_df['topic'] == topic]
    if len(subset) > target_per_class:
        # Даунсэмплинг для больших классов
        balanced_dfs.append(subset.sample(target_per_class, random_state=42))
    else:
        # Берем все данные для малых классов
        balanced_dfs.append(subset)

# Собираем и перемешиваем данные
balanced_df = pd.concat(balanced_dfs).sample(frac=1, random_state=42).reset_index(drop=True)

# Если суммарное количество строк меньше 100K, добавляем недостающее из "богатых" классов
current_size = len(balanced_df)
if current_size < 100_000:
    additional_samples = 100_000 - current_size
    # Выбираем классы, у которых есть избыток данных
    candidates = class_df.groupby('topic').filter(lambda x: len(x) > target_per_class)
    # Берем дополнительные образцы (без пересечения с уже выбранными)
    additional_df = candidates.sample(n=additional_samples, 
                                     replace=False, 
                                     random_state=42)
    # Финализируем датафрейм
    balanced_df = pd.concat([balanced_df, additional_df]).sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Итоговый размер: {len(balanced_df)} строк")

Итоговый размер: 100000 строк


In [6]:
balanced_df['topic'].value_counts()

topic
Россия               10904
Мир                  10126
Экономика             7898
Спорт                 7405
Наука и техника       7019
Бывший СССР           6987
Культура              6935
Интернет и СМИ        6678
Из жизни              6023
Дом                   5782
Силовые структуры     5757
Ценности              5299
Бизнес                5270
Путешествия           5261
69-я параллель        1268
Крым                   666
Культпросвет           340
EMPTY                  203
Легпром                114
Библиотека              65
Name: count, dtype: int64

In [9]:
balanced_df

,title,text,topic
0,В подмосковных Химках обстреляли перевозчиков ...,Полиция разыскивает напавших с травматическими...,Силовые структуры
1,"Премию Астрид Линдгрен вручили лауреату ""Оскара""","Лауреатом премии имени Астрид Линдгрен, котору...",Культура
2,O1 Properties заинтересовалась бизнес-центром ...,Инвесткомпания O1 Properties ведет переговоры ...,Дом
3,«Газпром» приступил к увольнениям,«Газпром» в течение года собирается сократить ...,Экономика
4,Медальон с прядью волос Джейн Остин выставили ...,Британский аукционный дом Dominic Winter выста...,Культура
...,...,...,...
99995,Астрономы обнаружили планетарные тостеры,"Астрономы предложили объяснение феномену ""разд...",Наука и техника
99996,Спецназовец оказался слишком женственным для Б...,"Бывшему военнослужащему армии США, офицеру вой...",Из жизни
99997,ЕС выделит Гаити 200 миллионов евро,Европейский союз выделит из своего бюджета 200...,Мир
99998,Суд выселил кота из коммунальной квартиры,В Кургане суд постановил выселить из коммуналь...,Из жизни


Решил проверить текст на html разметку и пробежаться по результату, что бы подобрать оптимальные паттерны для обработки текста

In [3]:
import re

def check_html_in_column(dframe: pd.DataFrame, column: str, sample_size: int = 5) -> dict:
    """
    Анализирует колонку DataFrame на наличие HTML-тегов
    """
    html_pattern = re.compile(r'<[^>]+>')
    
    mask = dframe[column].apply(
        lambda x: bool(html_pattern.search(str(x))) if pd.notnull(x) else False
    )
    
    total_rows = len(df)
    html_rows = mask.sum()
    
    result = {
        'total_rows': total_rows,
        'html_rows': html_rows,
        'html_percent': round(html_rows / total_rows * 100, 2) if total_rows > 0 else 0,
        'examples': dframe[column][mask].sample(min(sample_size, html_rows)).tolist() if html_rows > 0 else []
    }
    
    if result['html_rows'] > 0:
        print("\nПримеры строк с HTML:")
        for example in result['examples']:
            print(f"- {example}...")

check_html_in_column(df, 'text', sample_size=3)


Примеры строк с HTML:
- Реальная доходность по банковским депозитам россиян в 2011 году будет отрицательной, если рост кредитных портфелей отечественных банков не превысит 5-7 процентов в год. Такое заявление сделал заместитель гендиректора Агентства по страхованию вкладов (АСВ) Андрей Мельников, передает "Интерфакс". По всей видимости, Мельников имел в виду отрицательную доходность с учетом инфляции. По словам Мельникова, в случае низкой кредитной активности банки не смогут компенсировать высокие ставки по вкладам, и, следовательно, им придется эти ставки снижать. В то же время в АСВ ожидают, что даже в таких условиях подавляющее большинство населения продолжит "нести деньги в банки". За первые шесть месяцев 2010 года объем вкладов россиян увеличился на 12,7 процента и превысил 8,4 триллиона рублей. 25 августа в ЦБ сообщили, что за июль объем депозитов населения в банках России вырос на 2,1 процента и составил более 8,6 триллиона рублей. Согласно прогнозу АСВ, рост вкладов физических

Для работы с venv

cd ~/.virtualenvs/csu_nlp_course_hw_1

source bin/activate

python -m spacy download ru_core_news_sm

pip install lxml

In [ ]:
!python -m spacy download ru_core_news_sm

In [7]:
import pandas as pd
import spacy
import re
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

class TextProcessing:
    def __init__(self, batch_size=100, n_threads=8):
        self.batch_size = batch_size
        self.n_threads = n_threads
        self.nlp = spacy.load("ru_core_news_sm", disable=["parser", "ner"])

    def _clean_text(self, text):
        # Удаляем знаки препинания и цифры
        return re.sub(r'[^\w\s]', ' ', text)

    def _process_batch(self, texts):
        cleaned_texts = [self._clean_text(text) for text in texts]
        lemmatized_texts = []

        for doc in self.nlp.pipe(cleaned_texts, batch_size=self.batch_size):
            lemmas = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.is_alpha]
            lemmatized_texts.append(' '.join(lemmas))

        return lemmatized_texts

    def process_texts(self, text_series: pd.Series) -> pd.Series:
        texts = text_series.tolist()
        total = len(texts)
        batch_size = self.batch_size

        results = [None] * total

        def process_and_store(start_idx, end_idx):
            batch = texts[start_idx:end_idx]
            processed_batch = self._process_batch(batch)
            return start_idx, processed_batch

        futures = []
        with ThreadPoolExecutor(max_workers=self.n_threads) as executor:
            for start_idx in range(0, total, batch_size):
                end_idx = min(start_idx + batch_size, total)
                futures.append(executor.submit(process_and_store, start_idx, end_idx))

            with tqdm(total=len(futures), desc="Processing batches", leave=True) as pbar:
                for future in as_completed(futures):
                    start_idx, processed_batch = future.result()
                    results[start_idx:start_idx+len(processed_batch)] = processed_batch
                    pbar.update(1)

        return pd.Series(results, index=text_series.index)
    
    
# df_head = balanced_df.head(2000).copy()
processor = TextProcessing(batch_size=200, n_threads=4)
# df_head["processed_text"] = processor.process_texts(df_head["text"])
# balanced_df["processed_text"] = processor.process_texts(balanced_df["text"])

In [7]:
balanced_df["processed_text"] = processor.process_texts(balanced_df["text"])
balanced_df["processed_title"] = processor.process_texts(balanced_df["title"])
balanced_df.to_csv("processed_balanced_text.csv", index=False, encoding='utf-8')

Processing batches:   0%|          | 0/500 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/500 [00:00<?, ?it/s]

### Разбор pipeline

Начать все же хотелось с того, что одно слово из за склонений может иметь множество вариаций, которые могут запутать алгоритм, ведь тогда в его словаре будет допустим 3 слова вместо одного. Для того что бы исправить эту проблему я выбрал библиотеку `spacy `и модель `ru_core_news_sm`, к сожалению мне не удалось установить `spacy[cuda]` и пришлось работать на процессоре, в результате для ускорения производительности был выбран `spacy.pipe` для обработки сразу списка, для ускорения обработка Series запускается в много потоке и на каждый поток выделяется определенный размер batсh что позволило в 3-4 раза ускорить обработку. Для того что бы оптимизировать и ускорить работу лематтизатора, текст был очищен от лишних знаков. В результате чего появился явный минус, что названия компаний или имена разделяются на разные слова, но я и так опаздываю и потратил очень много времени на оптимизацию процесса обработки текста, я не успеваю(.    

In [8]:
from sklearn.model_selection import train_test_split

100000 / len(class_df)
split_df, empty_df = train_test_split(
    class_df, train_size=100000 / len(class_df), stratify=class_df['topic']
)

empty_df = pd.DataFrame()

print(len(split_df))

processor = TextProcessing(batch_size=200, n_threads=4)
split_df["processed_text"] = processor.process_texts(split_df["text"])
split_df["processed_title"] = processor.process_texts(split_df["title"])
split_df.to_csv("processed_split_df.csv", index=False, encoding='utf-8')

100000


Processing batches:   0%|          | 0/500 [00:00<?, ?it/s]

Processing batches:   0%|          | 0/500 [00:00<?, ?it/s]

In [4]:
split_df['topic'].value_counts()

topic
Россия               21711
Мир                  18487
Экономика            10758
Спорт                 8713
Культура              7277
Бывший СССР           7223
Наука и техника       7187
Интернет и СМИ        6043
Из жизни              3735
Дом                   2940
Силовые структуры     2650
Ценности              1050
Бизнес                1001
Путешествия            867
69-я параллель         171
Крым                    90
Культпросвет            46
EMPTY                   27
Легпром                 15
Библиотека               9
Name: count, dtype: int64

Импортирование обработанных датасетов

In [4]:
import pandas as pd
balanced_df = pd.read_csv("processed_balanced_text.csv", encoding='utf-8')
split_df = pd.read_csv("processed_split_df.csv", encoding='utf-8')

Объединяем обработанный текст. И собираем словарь фреймов для удобства.

In [5]:
from sklearn.model_selection import train_test_split

balanced_df.loc[:, 'processed_full_text'] = balanced_df['processed_text'].astype(str) + ' ' + balanced_df['processed_title'].astype(str)
split_df.loc[:, 'processed_full_text'] = split_df['processed_text'].astype(str) + ' ' + split_df['processed_title'].astype(str)

df_balanced_cleaned = balanced_df.loc[~balanced_df['topic'].isin(['EMPTY', 'Библиотека', "Легпром"])].copy()
df_split_cleaned = split_df.loc[~split_df['topic'].isin(['EMPTY', 'Библиотека', "Легпром"])].copy()


data_dfs = {
    "balanced_df" : {"df": balanced_df},
    "split_df" : {"df": split_df},
    "df_balanced_cleaned" : {"df": df_balanced_cleaned},
    "df_split_cleaned" : {"df": df_split_cleaned}
}

for name, data in data_dfs.items():
    frame = data["df"]

    train_df, temp_df = train_test_split(
        frame, test_size=0.4, stratify=frame['topic']
    )
    
    val_df, test_df = train_test_split(
        temp_df, test_size=0.5, stratify=temp_df['topic']
    )
    
    data["train_df"] = train_df
    data["val_df"] = val_df
    data["test_df"] = test_df


In [6]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

for name, data in data_dfs.items():
    
    train_df = data["train_df"]
    val_df = data["val_df"]

    X_train = train_df['processed_full_text']
    y_train = train_df['topic']
    
    X_val = val_df['processed_full_text']
    y_val = val_df['topic']
    
    dummy_clf = DummyClassifier(strategy="most_frequent", random_state=42)
    dummy_clf.fit(X_train, y_train)
    y_val_pred = dummy_clf.predict(X_val)
    
    accuracy = accuracy_score(y_val, y_val_pred)
    print(f"Базовое качество DummyClassifier {name} (most_frequent): {accuracy:.4f}")

Базовое качество DummyClassifier balanced_df (most_frequent): 0.1090
Базовое качество DummyClassifier split_df (most_frequent): 0.2171
Базовое качество DummyClassifier df_balanced_cleaned (most_frequent): 0.1095
Базовое качество DummyClassifier df_split_cleaned (most_frequent): 0.2173


Проверяем фреймы на пустые значения и получаем, что Proccess текст уничтожил одну строку текст, уберем её и пойдем дальше.

In [7]:
print(split_df.isna().any().any())
print(balanced_df.isna().any().any())
print(df_balanced_cleaned.isna().any().any())
print(df_split_cleaned.isna().any().any())


True
False
False
True


In [8]:
split_df.dropna(inplace=True)
balanced_df.dropna(inplace=True)
df_balanced_cleaned.dropna(inplace=True)
df_split_cleaned.dropna(inplace=True)

In [45]:
print(split_df['topic'].value_counts())
print(balanced_df['topic'].value_counts())

topic
Россия               21710
Мир                  18487
Экономика            10758
Спорт                 8713
Культура              7277
Бывший СССР           7223
Наука и техника       7187
Интернет и СМИ        6043
Из жизни              3735
Дом                   2940
Силовые структуры     2650
Ценности              1050
Бизнес                1001
Путешествия            867
69-я параллель         171
Крым                    90
Культпросвет            46
EMPTY                   27
Легпром                 15
Библиотека               9
Name: count, dtype: int64
topic
Россия               10904
Мир                  10126
Экономика             7898
Спорт                 7405
Наука и техника       7019
Бывший СССР           6987
Культура              6935
Интернет и СМИ        6678
Из жизни              6023
Дом                   5782
Силовые структуры     5757
Ценности              5299
Бизнес                5270
Путешествия           5261
69-я параллель        1268
Крым             

Проверяем что мы не потеряли ни один классификатор во всех фреймах, которые будем использовать для обучения.

In [53]:
print(len(data_dfs["balanced_df"]["test_df"]['topic'].value_counts()))
print(len(data_dfs["balanced_df"]["train_df"]['topic'].value_counts()))
print(len(data_dfs["balanced_df"]["val_df"]['topic'].value_counts()))
print(len(data_dfs["split_df"]["test_df"]['topic'].value_counts()))
print(len(data_dfs["split_df"]["train_df"]['topic'].value_counts()))
print(len(data_dfs["split_df"]["val_df"]['topic'].value_counts()))

20
20
20
20
20
20


Подготовив все данные приступлю к обучению модели `LogisticRegression`. Для настроек `TfidfVectorizer` я выбрал триграммы, потому что работаем с большим количеством документов и со среднем объёмом текстов. Я не стал устанавливать параметр max_features, потому что хоть это и может значительно ускорить обучение, но я выполнил лематизацию и урезание матрицы при помощи max_features нигативно сказывается на результате, оптимально минимальным параметром можно выставить значение 10000, на моих данных данное значение параметра показывает максимальный результат между скоростью и качеством, в скорости 2-3 раза, в качестве, 0,15. Так же из-за низкочисленных классификаторов таких как "EMPTY" и "Библиотека" необходимо установить умеренную регуляризацию `C=10`, иначе модель отказывается обучиться и показывает f1 0 на этих классификаторах.

Использование ngram_range=(1, 3) позволило модели учитывать контекст. Параметры min_df=5 и max_df=0.9 помогли отфильтровать редкие и слишком частые токены, что также улучшило результаты.

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

dataframe_train = data_dfs["balanced_df"]["train_df"]
dataframe_val = data_dfs["balanced_df"]["val_df"]
dataframe_test = data_dfs["balanced_df"]["test_df"]

y_train = dataframe_train['topic']
y_val = dataframe_val['topic']
y_test = dataframe_test['topic']

vectorizer = TfidfVectorizer(
        ngram_range=(1, 3),
        min_df=5,
        max_df=0.9,
        )

X_train_vect = vectorizer.fit_transform(dataframe_train['processed_full_text'])
X_val_vect = vectorizer.transform(dataframe_val['processed_full_text'])
X_test_vect = vectorizer.transform(dataframe_test['processed_full_text'])

model = LogisticRegression(
        max_iter=10000, # ConvergenceWarning
        C = 10,
        n_jobs=-1,
        )

model.fit(X_train_vect, y_train)

# Проверка на валидации
y_val_pred = model.predict(X_val_vect)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"\nКачество LogisticRegression с TfidfVectorizer на валидации: {val_accuracy:.4f}")

# Проверка на тестовой выборке
y_test_pred = model.predict(X_test_vect)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Качество LogisticRegression с TfidfVectorizer на тестовой выборке: {test_accuracy:.4f}")

# Отчёт по классам
print("\nClassification report на тесте с TfidfVectorizer:")
print(classification_report(y_test, y_test_pred))


Качество LogisticRegression с TfidfVectorizer на валидации: 0.8183
Качество LogisticRegression с TfidfVectorizer на тестовой выборке: 0.8154

Classification report на тесте с TfidfVectorizer:
                   precision    recall  f1-score   support

   69-я параллель       0.90      0.73      0.81       254
            EMPTY       1.00      0.20      0.33        40
       Библиотека       1.00      0.31      0.47        13
           Бизнес       0.78      0.78      0.78      1054
      Бывший СССР       0.86      0.84      0.85      1397
              Дом       0.89      0.88      0.88      1156
         Из жизни       0.70      0.71      0.71      1205
   Интернет и СМИ       0.79      0.76      0.77      1336
             Крым       0.77      0.65      0.70       133
    Культпросвет        0.88      0.53      0.66        68
         Культура       0.85      0.87      0.86      1387
          Легпром       0.83      0.43      0.57        23
              Мир       0.74      0.79 

Предположив, что EMPTY является классификатором, который обозначаниет что у статьи нету темы, я решил его убрать и так же убрать низкочисленные классификаторы такие как "Легпром" и "Библиотека". Качество предсказания модели улучшилось на 0,1 при `С=1`

In [54]:
dataframe_train = data_dfs["df_balanced_cleaned"]["train_df"]
dataframe_val = data_dfs["df_balanced_cleaned"]["val_df"]
dataframe_test = data_dfs["df_balanced_cleaned"]["test_df"]

y_train = dataframe_train['topic']
y_val = dataframe_val['topic']
y_test = dataframe_test['topic']

vectorizer = TfidfVectorizer(
        ngram_range=(1, 3),
        min_df=5,
        max_df=0.9,
        )

X_train_vect = vectorizer.fit_transform(dataframe_train['processed_full_text'])
X_val_vect = vectorizer.transform(dataframe_val['processed_full_text'])
X_test_vect = vectorizer.transform(dataframe_test['processed_full_text'])

model = LogisticRegression(
        n_jobs=-1,
        max_iter=10000, # ConvergenceWarning
        )

model.fit(X_train_vect, y_train)

# Проверка на валидации
y_val_pred = model.predict(X_val_vect)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"\nКачество LogisticRegression с TfidfVectorizer на валидации: {val_accuracy:.4f}")

# Проверка на тестовой выборке
y_test_pred = model.predict(X_test_vect)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Качество LogisticRegression с TfidfVectorizer на тестовой выборке: {test_accuracy:.4f}")

# Отчёт по классам
print("\nClassification report на тесте с TfidfVectorizer:")
print(classification_report(y_test, y_test_pred))


Качество LogisticRegression с TfidfVectorizer на валидации: 0.8072
Качество LogisticRegression с TfidfVectorizer на тестовой выборке: 0.8056

Classification report на тесте с TfidfVectorizer:
                   precision    recall  f1-score   support

   69-я параллель       0.93      0.66      0.77       253
           Бизнес       0.81      0.76      0.78      1054
      Бывший СССР       0.82      0.85      0.83      1397
              Дом       0.86      0.84      0.85      1157
         Из жизни       0.70      0.70      0.70      1205
   Интернет и СМИ       0.77      0.76      0.77      1335
             Крым       0.81      0.59      0.69       133
    Культпросвет        0.74      0.25      0.37        68
         Культура       0.85      0.88      0.86      1387
              Мир       0.74      0.79      0.77      2026
  Наука и техника       0.83      0.82      0.83      1404
      Путешествия       0.86      0.83      0.85      1052
           Россия       0.69      0.76 

# Подбор параметров

Начнем подбор парметров. C=10, C=50: Постепенное уменьшение регуляризации, чтобы модель могла лучше подстроиться под данные. max_iter=10000 и max_iter=15000 — это большие значения, которые гарантируют, что модель успеет сойтись даже на сложных данных, потому что 1000 и 5000 не хватает и в малочисленных классификаторах я получаю f1 = 0. Использую `balanced` из за, что данные несбалансированы, хоть я и попытался равномерно их распределить, но все равно этого не достаточно, что бы назвать его сбалансированным. 

Оставил `penalty=l2`, потому что `solver=lbfgs` поддерживает только его. ``lbfgs`` хорошо подходит для данных с умеренным количеством признаков и более быстрый.

Так же у меня к сожалению не получилось подобрать параметры к saga, хоть он и должен показывать себя лучше на больших объёмах данных, но я прожал 3 часа, но так и получил результат по `GridSearchCV`.

Так же я установил `max_features=10000`, что конечно повлияет на результат, но зачительно ускорит подборк.

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

def grid_search(vectorizer, data):
    param_grid = {
    'C': [10, 50],
    'max_iter': [10000, 7500, 12500],
    'penalty': ['l2'],
    'solver': ['lbfgs'],
    'class_weight': ['balanced'],
    }   

    logreg = LogisticRegression(
        random_state=42,
        n_jobs=-1
    )
    
    grid_search = GridSearchCV(
        logreg,
        param_grid,
        cv=3,  # кросс-валидация по 3 фолдам, ну чуть быстрее.
        scoring='accuracy',
        n_jobs=-1,
        verbose=2
    )
    
    dataframe_train = data["train_df"]
    dataframe_val = data["val_df"]
    dataframe_test = data["test_df"]
    
    y_train = dataframe_train['topic']
    y_val = dataframe_val['topic']
    y_test = dataframe_test['topic']

    X_train_vect = vectorizer.fit_transform(dataframe_train['processed_full_text'])
    X_val_vect = vectorizer.transform(dataframe_val['processed_full_text'])
    X_test_vect = vectorizer.transform(dataframe_test['processed_full_text'])
    
    grid_search.fit(X_train_vect, y_train)
    
    print(f"\nЛучшие параметры: {grid_search.best_params_}")
    print(f"Лучшее качество на кросс-валидации: {grid_search.best_score_:.4f}")
    
    # Оценка на валидации с лучшей моделью
    best_model = grid_search.best_estimator_
    y_val_pred = best_model.predict(X_val_vect)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    print(f"\nКачество на валидации с лучшими параметрами: {val_accuracy:.4f}")
    
    # Финальная проверка на тесте
    y_test_pred = best_model.predict(X_test_vect)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"\nКачество на тестовой выборке: {test_accuracy:.4f}")
    
    # Отчёт по классам
    print("\nClassification report на тесте:")
    print(classification_report(y_test, y_test_pred))
    
grid_search(TfidfVectorizer(
        max_features=10000,
        ngram_range=(1, 3),
        min_df=5,
        max_df=0.9,
        ), data_dfs["balanced_df"])

    

Fitting 3 folds for each of 6 candidates, totalling 18 fits

Лучшие параметры: {'C': 10, 'class_weight': 'balanced', 'max_iter': 10000, 'penalty': 'l2', 'solver': 'lbfgs'}
Лучшее качество на кросс-валидации: 0.8010

Качество на валидации с лучшими параметрами: 0.8063

Качество на тестовой выборке: 0.8105

Classification report на тесте:
                   precision    recall  f1-score   support

   69-я параллель       0.83      0.86      0.85       253
            EMPTY       0.35      0.17      0.23        40
       Библиотека       1.00      0.85      0.92        13
           Бизнес       0.76      0.79      0.78      1054
      Бывший СССР       0.83      0.86      0.85      1397
              Дом       0.88      0.87      0.88      1157
         Из жизни       0.68      0.73      0.70      1205
   Интернет и СМИ       0.78      0.77      0.78      1336
             Крым       0.69      0.80      0.75       133
    Культпросвет        0.58      0.66      0.62        68
         Ку

In [18]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from tqdm.notebook import tqdm

# Настройки векторизаторов
vectorizers = {
    'CountVectorizer': "",
    'TfidfVectorizer': ""
}

results = {}

for name, vectorizer in tqdm(vectorizers.items(), desc="Векторизаторы"):
    if name == "CountVectorizer":
        vectorizer = CountVectorizer(
        
        ngram_range=(1, 3),
        min_df=5,
        max_df=0.9
    )
    else:
        vectorizer = TfidfVectorizer(
        
        ngram_range=(1, 3),
        min_df=5,
        max_df=0.9,
        sublinear_tf=True
    )
    print(f"\n{name}:")
    for name, data in data_dfs.items():
        print(f"frame: {name}")
        
        dataframe_train = data["train_df"]
        dataframe_val = data["val_df"]
        dataframe_test = data["test_df"]
        
        y_train = dataframe_train['topic']
        y_val = dataframe_val['topic']
        y_test = dataframe_test['topic']

        X_train_vect = vectorizer.fit_transform(dataframe_train['processed_full_text'])
        X_val_vect = vectorizer.transform(dataframe_val['processed_full_text'])
        X_test_vect = vectorizer.transform(dataframe_test['processed_full_text'])
    
        model = LogisticRegression(
            C=10,
            penalty='l2',
            class_weight='balanced',
            max_iter=10000,
            solver='lbfgs',
            n_jobs=-1
        )
        model.fit(X_train_vect, y_train)
    
        # Предсказания
        y_val_pred = model.predict(X_val_vect)
        y_test_pred = model.predict(X_test_vect)
    
        # Метрики
        val_accuracy = accuracy_score(y_val, y_val_pred)
        test_accuracy = accuracy_score(y_test, y_test_pred)
        
        print(f"  Валидация: {val_accuracy:.4f}")
        print(f"  Тест: {test_accuracy:.4f}")
        
        # Сохраняем результаты
        results[name] = {
            'val_accuracy': val_accuracy,
            'test_accuracy': test_accuracy,
            'classification_report': classification_report(y_test, y_test_pred, output_dict=True, zero_division=0)
        }

Векторизаторы:   0%|          | 0/2 [00:00<?, ?it/s]


CountVectorizer:
frame: balanced_df
  Валидация: 0.8092
  Тест: 0.8113
frame: split_df
  Валидация: 0.8102
  Тест: 0.8139
frame: df_balanced_cleaned
  Валидация: 0.8134
  Тест: 0.8169
frame: df_split_cleaned
  Валидация: 0.8131
  Тест: 0.8165

TfidfVectorizer:
frame: balanced_df
  Валидация: 0.8286
  Тест: 0.8306
frame: split_df
  Валидация: 0.8293
  Тест: 0.8268
frame: df_balanced_cleaned
  Валидация: 0.8324
  Тест: 0.8339
frame: df_split_cleaned
  Валидация: 0.8286
  Тест: 0.8312


`TfidfVectorizer` показал себя лучше, чем `CountVectorizer`, во всех случаях. Это связано с тем, что TfidfVectorizer учитывает не только частоту слов, но и их важность в контексте всего документа, что позволяет лучше выделять значимые признаки.

Попытка равномерно распределить данные (balanced_df) показала результаты, близкие к несбалансированным (split_df), но после удаления классификаторов с низкой значимостью качество модели немного улучшилось.


balanced_df: accuracy на тесте — 0.8306.
df_balanced_cleaned: accuracy на тесте — 0.8339.

В итоге удаление малоинформативных классов помогает модели лучше обобщать, конечно не значительно, но результат есть.

Модель демонстрирует стабильность на валидационной и тестовой выборках, что говорит об отсутствии переобучения.

Очистка данных и более равномерное распределение классов положительно влияют на качество модели.